In [1]:
%%time
! python3 -m pip install --upgrade sagemaker
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role
print(f"SageMaker Execution Role:{role}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account:{account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region:{region}")

     |████████████████████████████████| 511 kB 34.9 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.75.1-py2.py3-none-any.whl size=709437 sha256=9944f257dbf4f3d6b9fa68214fe7dff7e1fa68c63af8739fe0d5d23adea76dcf
  Stored in directory: /root/.cache/pip/wheels/04/4e/04/ad25d8866c9738524d659f5e88015dba26ebd28ced7bb8c799
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.72.0
    Uninstalling sagemaker-2.72.0:
      Successfully uninstalled sagemaker-2.72.0
SageMaker Execution Role:arn:aws:iam::900051432098:role/service-role/AmazonSageMaker-ExecutionRole-20210311T111844
AWS account:900051432098
AWS region:ap-southeast-1
CPU times: user 992 ms, sys: 292 ms, total: 1.28 s
Wall time: 6.1 s


## Build and Push Docker image to ECR

In [9]:
image = "bert-smdataparallel-sagemaker"
tag = "pt1.8"

In [8]:
!wget "https://raw.githubusercontent.com/xjlwi/amazon-sagemaker-examples/main/training/distributed_training/pytorch/data_parallel/bert/train.sh"

--2022-02-12 04:45:32--  https://raw.githubusercontent.com/xjlwi/amazon-sagemaker-examples/main/training/distributed_training/pytorch/data_parallel/bert/train.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 621 [text/plain]
Saving to: ‘train.sh’

train.sh            100%[===================>]     621  --.-KB/s    in 0s      

2022-02-12 04:45:32 (37.0 MB/s) - ‘train.sh’ saved [621/621]



In [19]:
!python3 -m pip install jedi==0.17.2
!pygmentize ./Dockerfile

     |████████████████████████████████| 1.4 MB 39.8 MB/s            
     |████████████████████████████████| 109 kB 122.8 MB/s            
  Attempting uninstall: parso
    Found existing installation: parso 0.8.0
    Uninstalling parso-0.8.0:
      Successfully uninstalled parso-0.8.0
  Attempting uninstall: jedi
    Found existing installation: jedi 0.18.0
    Uninstalling jedi-0.18.0:
      Successfully uninstalled jedi-0.18.0
ARG region

FROM 763104351884.dkr.ecr.${region}.amazonaws.com/pytorch-training:1.8.1-gpu-py36-cu111-ubuntu18.04

ARG WORK_DIR="apex_build"
RUN pip --no-cache-dir --no-cache install h5py boto3 'git+https://github.com/NVIDIA/dllogger' tqdm requests; \
      cd $WORK_DIR; \
      git clone https://github.com/NVIDIA/apex; cd apex; \
      python setup.py install --cuda_ext --cpp_ext; \
      cd ../..; rm -rf $WORK_DIR;


In [20]:
!pygmentize ./build_and_push.sh

#!/usr/bin/env bash
# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.
# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# set region

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"

if [ "$#" -eq 3 ]; then
    region=$1
    image=$2
    tag=$3
else
    echo "usage: $0 <aws-region> $1 <image-repo> $2 <image-tag>"
    exit 1
fi
# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)
if [ $? -ne 0 ]
then
    exit 255
fi

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"
# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    aws ecr create-repository --region ${region} --repositor

## Tensorflow Board

In [2]:
import os
import datetime
import tensorflow as tf

In [3]:
# create a keras model
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

11493376/11490434 [==============================] - 1s 0us/step


In [4]:
#Create a directory for your TensorBoard logs
LOG_DIR = os.path.join(os.getcwd(), "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))


In [5]:
# Run training 

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
              
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)

model.fit(x=x_train,
          y=y_train,
          epochs=5,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

[2022-02-12 05:17:42.746 tensorflow-2-3-gpu--ml-g4dn-xlarge-8248464766bb46d9e489eb7b7a82:77 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-02-12 05:17:42.771 tensorflow-2-3-gpu--ml-g4dn-xlarge-8248464766bb46d9e489eb7b7a82:77 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Epoch 1/5
   1/1875 [..............................] - ETA: 0s - loss: 2.4083 - accuracy: 0.0312WARNING:tensorflow:From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


   2/1875 [..............................] - ETA: 2:20 - loss: 2.3173 - accuracy: 0.1250WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0061s vs `on_train_batch_end` time: 0.1436s). Check your callbacks.


1875/1875 [==============================] - 4s 2ms/step - loss: 0.2252 - accuracy: 0.9336 - val_loss: 0.1211 - val_accuracy: 0.9624
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0981 - accuracy: 0.9701 - val_loss: 0.0731 - val_accuracy: 0.9770
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0695 - accuracy: 0.9786 - val_loss: 0.0760 - val_accuracy: 0.9747
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0534 - accuracy: 0.9830 - val_loss: 0.0645 - val_accuracy: 0.9800
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0437 - accuracy: 0.9855 - val_loss: 0.0605 - val_accuracy: 0.9823


In [6]:
EFS_PATH_LOG_DIR = "/".join(LOG_DIR.strip("/").split('/')[1:-1])
print (EFS_PATH_LOG_DIR)

sagemaker-r/sm-r/notebooks/logs/fit
